# **hkl_soleil** E6C $\psi$ (psi) axis

- **TODO: Switch from simulated positioners to EpicsMotor positioners**
- FIXME: Why cannot e6c_psi.move(25)?  (Or any float value within limits)
- TODO: Is `e6c_psi` actually needed?  (Probably not, if can't scan it.)

Show how to set, compute, and scan $\psi$ with the E6C diffractometer geometry
using the `hkl_soleil` solver.  Scan $\psi$ at fixed $Q$ and $hkl_2$.

Virtual axes, such as $\psi$, are features provided by the solver as *extras*.
*Extras* are not necessarily available in every solver.  Consult the solver
documentation for details.

NOTE: The demonstrations below rely on features provided by the `hkl_soleil`
solver.

## Overview

To set and compute $\psi$, we'll use two instances of the E6C diffractometer
geometry, each with a different calculation engine, described in the next table.

engine | how it is used
--- | ---
`"hkl"` | work in reciprocal-space coordinates $h, k, l$
`"psi"` | work with the $\psi$ rotation

<!--
TODO: compare with APS POLAR geometry

See J. Appl. Cryst. (1999). 32, 943-950, https://doi.org/10.1107/S0021889899007347
"Angle calculations for a six-circle κ diffractometer"
G. Thorkildsen, R. H. Mathiesen and H. B. Larsen

Also, DLS has a K6C at I16
https://indico.psi.ch/event/12738/contributions/38939/attachments/22657/39873/NOBUGS2022_diffcalc_v2.pdf
-->

$\psi$ is the rotation of reference vector $hkl_2$ perpendicular to scattering
vector $Q$:

<img src="../_static/psi-angle.png" width="400" />

color | description
--- | ---
blue | incident and exit X-ray beams
green | scattering vector ($Q$)
red | reference vector ($hkl_2$)
yellow | rotation ($\psi$) from $hkl_2$ around $Q$
black | principle cartesian axes
gray | sample

## Steps

1. With the `"hkl"` engine:
    1. Orient a crystalline sample with the `"hkl"` engine.
    1. Define the azimuthal reflection $h_2, k_2, l_2$ and a $\psi$ rotation around
    that azimuthal reflection.
    1. Position the diffractometer for the $h, k, l$ reflection.
1. With the `"psi"` engine:
    1. Copy sample, orientation, and position information from the `"hkl"` instance.
    1. Compare the computed `psi` value with the value we set previously.
1. Scan $\psi$ at fixed $Q$ and $hkl_2$.

## SimulatedE6C

Create instances of (simulated) E6C for the `"hkl"` and `"psi"` solver engines.
Pre-defined simulator classes are available for both engines.

In [1]:
from hklpy2 import SimulatedE6C, SimulatedE6C_Psi

e6c_hkl = SimulatedE6C(name="e6c_hkl")
e6c_psi = SimulatedE6C_Psi(name="e6c_psi")

Show the different calculation engines available for the E6C geometry.

In [2]:
print(f"{e6c_hkl.operator.solver.engines=}")

e6c_hkl.operator.solver.engines=['hkl', 'psi', 'q2', 'qper_qpar', 'tth2', 'incidence', 'emergence']


Show the different operation modes available with each engine for the E6C geometry.

The `hkl` engine has a `"psi_constant_vertical"` mode that can be used to calculate reals given some fixed parameters (UB, wavelength, $(hkl)$, $(hkl)_2$, $\psi$).  The `psi` engine has only one mode.

In [3]:
print(f"{e6c_hkl.operator.solver.modes=}")
print(f"{e6c_psi.operator.solver.modes=}")

e6c_hkl.operator.solver.modes=['bissector_vertical', 'constant_omega_vertical', 'constant_chi_vertical', 'constant_phi_vertical', 'lifting_detector_phi', 'lifting_detector_omega', 'lifting_detector_mu', 'double_diffraction_vertical', 'bissector_horizontal', 'double_diffraction_horizontal', 'psi_constant_vertical', 'psi_constant_horizontal', 'constant_mu_horizontal']
e6c_psi.operator.solver.modes=['psi_vertical']


Show the extra axes available with each mode used by this notebook.

The `psi` engine has a pseudo axis `"psi"` that can be used to calculate $\psi$ given some fixed parameters (reals, UB, wavelength, $(hkl)$, $(hkl)_2$)

In [4]:
e6c_hkl.operator.solver.mode = "bissector_vertical"
print(f"{e6c_hkl.operator.solver.mode=}")
print(f"{e6c_hkl.operator.solver.extras=}")

e6c_hkl.operator.solver.mode = "psi_constant_vertical"
print(f"{e6c_hkl.operator.solver.mode=}")
print(f"{e6c_hkl.operator.solver.extras=}")

# "psi" engine has only one mode, do not need to set it
print(f"{e6c_psi.operator.solver.mode=}")
print(f"{e6c_psi.operator.solver.extras=}")

e6c_hkl.operator.solver.mode='bissector_vertical'
e6c_hkl.operator.solver.extras={}
e6c_hkl.operator.solver.mode='psi_constant_vertical'
e6c_hkl.operator.solver.extras={'h2': 1.0, 'k2': 0.0, 'l2': 0.0, 'psi': 0.0}
e6c_psi.operator.solver.mode='psi_vertical'
e6c_psi.operator.solver.extras={'h2': 1.0, 'k2': 1.0, 'l2': 1.0}


## Define and orient a sample

The sample for this notebook is crystalline vibranium, with a cubic lattice of exactly $2\pi$.  With it mounted on oru diffractometer, we have identified two reflections which define its orientation.

In [5]:
import math

# e6c_hkl.wavelength.put(1.54)  # angstrom (8.0509 keV)

e6c_hkl.add_sample("vibranium", 2 * math.pi, digits=5)

e6c_hkl.add_reflection((4, 0, 0), (0, 29.354, 0, 2, 0, 58.71), name="r400")
e6c_hkl.add_reflection((0, 4, 0), (0, 29.354, 0, 92, 0, 58.71), name="r040")
for r in e6c_hkl.sample.reflections.order:
    print(f"{e6c_hkl.sample.reflections[r]}")
e6c_hkl.operator.calc_UB(*e6c_hkl.sample.reflections.order)

print(f"{e6c_hkl.operator.solver.UB=!r}")
print(f"{e6c_hkl.operator.solver.U=!r}")


Reflection(name='r400', geometry='E6C', pseudos={'h': 4, 'k': 0, 'l': 0}, reals={'mu': 0, 'omega': 29.354, 'chi': 0, 'phi': 2, 'gamma': 0, 'delta': 58.71}, wavelength=1.0, digits=4)
Reflection(name='r040', geometry='E6C', pseudos={'h': 0, 'k': 4, 'l': 0}, reals={'mu': 0, 'omega': 29.354, 'chi': 0, 'phi': 92, 'gamma': 0, 'delta': 58.71}, wavelength=1.0, digits=4)
e6c_hkl.operator.solver.UB=[[0.034882054037, 0.999391435978, -0.0], [0.0, 0.0, 1.0], [0.999391435978, -0.034882054037, -0.0]]
e6c_hkl.operator.solver.U=[[0.034882054037, 0.999391435978, 0.0], [0.0, 0.0, 1.0], [0.999391435978, -0.034882054037, 0.0]]


## Move to the $(111)$ orientation

In [6]:
e6c_hkl.operator.solver.mode = "bissector_vertical"
e6c_hkl.move(1, 0, 0)
print(f"{e6c_hkl.position=}")
print(f"{e6c_hkl.real_position=}")

e6c_hkl.position=SimulatedE6CPseudoPos(h=1.000000000517, k=-7.014e-09, l=0)
e6c_hkl.real_position=SimulatedE6CRealPos(mu=0, omega=4.564279210432, chi=0, phi=1.998999598145, gamma=0, delta=9.128558420864)


## psi_constant_vertical mode -- extra axes

In [7]:
e6c_hkl.operator.solver.mode = "psi_constant_vertical"
print(f"{e6c_hkl.operator.solver.extra_axis_names=}")

e6c_hkl.operator.solver.extra_axis_names=['h2', 'k2', 'l2', 'psi']


Set azimuthal reflection $(110)$ and $\psi=12$.

In [8]:
e6c_hkl.operator.solver.extras = dict(h2=1, k2=1, l2=0, psi=12)
print(f"{e6c_hkl.operator.solver.extras=}")

e6c_hkl.operator.solver.extras={'h2': 1.0, 'k2': 1.0, 'l2': 0.0, 'psi': 12.0}


Compute the real-axis motor values with the $(111)$ reflection oriented and $\psi$ rotation around the azimuthal reflection.

In [9]:
p_111 = e6c_hkl.forward(1, 1, 1)
print(f"{p_111=}")

p_111=SimulatedE6CRealPos(mu=0, omega=62.059110004039, chi=99.773817896463, phi=-49.997332346514, gamma=0, delta=15.844851584345)


Move each real (real-space positioner) to the computed $(111)$ reflection position `p_111`.

In [10]:
e6c_hkl.move_reals(p_111)
print(f"{e6c_hkl.position=}")
print(f"{e6c_hkl.real_position=}")
print(f"{e6c_hkl.operator.solver.extras=}")

e6c_hkl.position=SimulatedE6CPseudoPos(h=0.99999999948, k=1.000000010605, l=1.000000009968)
e6c_hkl.real_position=SimulatedE6CRealPos(mu=0, omega=62.059110004039, chi=99.773817896463, phi=-49.997332346514, gamma=0, delta=15.844851584345)
e6c_hkl.operator.solver.extras={'h2': 1.0, 'k2': 1.0, 'l2': 0.0, 'psi': 12.0}


## Calculate $\psi$

In [11]:
print(f"{e6c_psi.operator.solver.mode=}")
print(f"{e6c_psi.operator.solver.extras=}")

e6c_psi.operator.solver.mode='psi_vertical'
e6c_psi.operator.solver.extras={'h2': 1.0, 'k2': 1.0, 'l2': 1.0}


Same sample and lattice

In [12]:
e6c_psi.add_sample("vibranium", 2 * math.pi, digits=5)

Sample(name='vibranium', lattice=Lattice(a=6.28319, system='cubic'))

Copy orientation from `hkl` instance.

In [13]:
e6c_psi.operator.solver.UB = e6c_hkl.operator.solver.UB

print(f"{e6c_psi.operator.solver.UB=!r}")
print(f"{e6c_psi.operator.solver.U=!r}")

e6c_psi.operator.solver.UB=[[0.034882112737, 0.999391462637, -7.7669e-08], [-1.1035e-07, 3.7043e-08, 0.999999954315], [0.999391567978, -0.034881973051, -8.4609e-08]]
e6c_psi.operator.solver.U=[[0.034882108064, 0.999391434092, -3.3171e-08], [-1.1035e-07, 3.7043e-08, 1.0], [0.999391434092, -0.034882108064, 1.11575e-07]]


Set ${hkl}_2=(1, 1, 0)$.

In [14]:
e6c_psi.operator.solver.extras = dict(h2=1, k2=1, l2=0)
print(f"{e6c_psi.operator.solver.extras=}")

e6c_psi.operator.solver.extras={'h2': 1.0, 'k2': 1.0, 'l2': 0.0}


Set real-axis position from `p_111` (above).

In [15]:
e6c_psi.move_reals(p_111)
print(f"{e6c_psi.pseudo_axis_names=}")
print(f"{e6c_psi.operator.solver.pseudo_axis_names=}")
print(f"{e6c_psi.position=}")
print(f"{e6c_psi.real_position=}")

e6c_psi.pseudo_axis_names=['psi']
e6c_psi.operator.solver.pseudo_axis_names=['psi']
e6c_psi.position=SimulatedE6C_PsiPseudoPos(psi=11.999992685501)
e6c_psi.real_position=SimulatedE6C_PsiRealPos(mu=0, omega=62.059110004039, chi=99.773817896463, phi=-49.997332346514, gamma=0, delta=15.844851584345)


Compare `hkl` and `psi` reports.

In [16]:
print(e6c_hkl)
e6c_hkl.wh()
print(e6c_psi)
e6c_psi.wh()

SimulatedE6C(prefix='', name='e6c_hkl', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['h', 'h.readback', 'h.setpoint', 'k', 'k.readback', 'k.setpoint', 'l', 'l.readback', 'l.setpoint', 'mu', 'omega', 'chi', 'phi', 'gamma', 'delta'], configuration_attrs=['h', 'k', 'l', 'geometry', 'solver', 'wavelength'], concurrent=True)
h=1.0 k=1.0 l=1.0
wavelength=1.0
mu=0 omega=62.0591 chi=99.7738 phi=-49.9973 gamma=0 delta=15.8449
h2=1.0 k2=1.0 l2=0 psi=12.0
SimulatedE6C_Psi(prefix='', name='e6c_psi', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['psi', 'psi.readback', 'psi.setpoint', 'mu', 'omega', 'chi', 'phi', 'gamma', 'delta'], configuration_attrs=['psi', 'geometry', 'solver', 'wavelength'], concurrent=True)
psi=12.0
wavelength=1.0
mu=0 omega=62.0591 chi=99.7738 phi=-49.9973 gamma=0 delta=15.8449
h2=1.0 k2=1.0 l2=0


## Scan

In [17]:
import databroker
import numpy

from bluesky import plans as bp
from bluesky import plan_stubs as bps
from bluesky import preprocessors as bpp
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
from ophyd.sim import noisy_det

from hklpy2 import ConfigurationRunWrapper
from hklpy2 import DiffractometerBase

# Save orientation of both diffractometers.
crw = ConfigurationRunWrapper(e6c_hkl, e6c_psi)

bec = BestEffortCallback()
bec.disable_plots()
cat = databroker.temp().v2
RE = RunEngine()
RE.subscribe(cat.v1.insert)
RE.subscribe(bec)
RE.preprocessors.append(crw.wrapper)

In [18]:
print(f"{e6c_psi.operator.solver.mode=}")
print(f"{e6c_psi.operator.solver.extras=}")

e6c_psi.operator.solver.mode='psi_vertical'
e6c_psi.operator.solver.extras={'h2': 1.0, 'k2': 1.0, 'l2': 0.0}


------

Added a diffractometer method `move_forward_with_extras(pseudos, extras)` to
enable scanning one of the axes in `extras`.

In [19]:
RE(e6c_hkl.move_forward_with_extras(dict(h=2, k=1, l=0), dict(h2=2, k2=2, l2=0, psi=0.1)))
e6c_hkl.wh()

h=2.0 k=1.0 l=0
wavelength=1.0
mu=0 omega=100.2499 chi=0.1 phi=-61.436 gamma=0 delta=20.4997
h2=2.0 k2=2.0 l2=0 psi=0.1


Build a custom plan that scans an extra axis, with the other extras and either
pseudos or reals fixed.

In [20]:
from hklpy2.operations.misc import dict_device_factory


def scan_extra(
    detectors: list,
    dfrct: DiffractometerBase,
    axis: str = None,  # name of extra parameter to be scanned
    start: float = None,
    finish: float = None,
    num: int = None,
    *,
    pseudos: dict = None,  # h, k, l
    reals: dict = None,  # angles
    extras: dict = {},  # define all but the 'axis', these will remain constant
    md: dict = None,
):
    """
    Scan one (or more) extra diffractometer parameter(s), such as psi.

    - iterate extras as decribed:
        - set extras
        - solution = forward(pseudos)
        - move to solution
        - trigger detectors
        - read all controls
    """
    if axis not in dfrct.operator.solver.extra_axis_names:
        raise KeyError(f"{axis!r} not in {dfrct.operator.solver.extra_axis_names}")
    if pseudos is None and reals is None:
        raise KeyError("Must define either pseudos or reals.")
    if pseudos is not None and reals is not None:
        raise KeyError("Cannot define both pseudos and reals.")

    forwardTransformation = reals is None

    _md = {
        "diffractometer": {
            "name": dfrct.name,
            "geometry": dfrct.operator.solver.geometry,
            "engine": dfrct.operator.solver.engine_name,
            "mode": dfrct.operator.solver.mode,
            "extra_axes": dfrct.operator.solver.extra_axis_names,
        },
        "axis": axis,
        "start": start,
        "finish": finish,
        "num": num,
        "pseudos": pseudos,
        "reals": reals,
        "extras": extras,
        "transformation": "forward" if forwardTransformation else "inverse",
    }.update(md or {})

    extras[axis] = start
    extras_class = dict_device_factory(extras)
    extras_device = extras_class("", name=f"{dfrct.name}_extras", kind="hinted")

    all_controls = detectors
    all_controls.append(extras_device)
    all_controls = list(set(all_controls))

    signal = getattr(extras_device, axis)  # Pick the 'axis' Component.
    signal.kind = "hinted"

    def position_series(start, finish, num):
        for value in numpy.linspace(start, finish, num=num):
            yield value

    @bpp.stage_decorator(detectors)
    @bpp.run_decorator(md=_md)
    def _inner():
        for value in position_series(start, finish, num):

            def move_axes(pseudos, reals, extras):
                """Move extras, then reals or pseudos, move to the solution."""
                if reals is None:
                    yield from dfrct.move_forward_with_extras(pseudos, extras)
                else:
                    # TODO: Inverse transformation
                    # yield from dfrct.inverse_move_with_extras(reals, extras)
                    raise NotImplementedError("Inverse transformation.")  # yet

            def acquire(objects):
                """Tell each object to acquire its data."""
                group = "trigger_control_objects"
                for item in objects:
                    yield from bps.trigger(item, group=group)
                yield from bps.wait(group=group)

            def record(objects, stream="primary"):
                """Read & record each object."""
                yield from bps.create(stream)
                for item in objects:
                    yield from bps.read(item)
                yield from bps.save()

            # note the new position for reporting later
            extras.update({axis: value})
            yield from bps.mv(signal, value)
            try:
                yield from move_axes(pseudos, reals, extras)
                yield from acquire(all_controls)
                yield from record(all_controls)
            except NotADirectoryError as reason:
                print(f"FAIL: {axis}={value} {reason}")

    return (yield from _inner())

Scan $\psi$ over a wide range in coarse steps.  This example chooses $Q=(210)$
and $hkl_2=(1 \bar2 0)$. (The reference $hkl_2$ was chosen to be perpendicular
to $Q$.)  Save the `uid` from the scan for later reference.

In [25]:
e6c_hkl.wh(full=True)  # diagnostic

e6c_hkl.operator.solver.mode = "psi_constant_vertical"
uid, = RE(
    scan_extra(
        [noisy_det, e6c_hkl],
        e6c_hkl,
        axis="psi",
        start=-175,
        finish=175,
        num=17,
        pseudos=dict(h=2, k=1, l=0),
        extras=dict(h2=1, k2=-2, l2=0),
    ),
)

e6c_hkl.wh(full=True)  # diagnostic

diffractometer='e6c_hkl'
HklSolver(name='hkl_soleil', version='5.1.2', geometry='E6C', engine_name='hkl', mode='psi_constant_vertical')
Sample(name='vibranium', lattice=Lattice(a=6.28319, system='cubic'))
U=[[0.034882054037, 0.999391435978, 0.0], [0.0, 0.0, 1.0], [0.999391435978, -0.034882054037, 0.0]]
UB=[[0.034882054037, 0.999391435978, -0.0], [0.0, 0.0, 1.0], [0.999391435978, -0.034882054037, -0.0]]
Reflection(name='r400', geometry='E6C', pseudos={'h': 4, 'k': 0, 'l': 0}, reals={'mu': 0, 'omega': 29.354, 'chi': 0, 'phi': 2, 'gamma': 0, 'delta': 58.71}, wavelength=1.0, digits=4)
Reflection(name='r040', geometry='E6C', pseudos={'h': 0, 'k': 4, 'l': 0}, reals={'mu': 0, 'omega': 29.354, 'chi': 0, 'phi': 92, 'gamma': 0, 'delta': 58.71}, wavelength=1.0, digits=4)
constraint: -180.0 <= mu <= 180.0
constraint: -180.0 <= omega <= 180.0
constraint: -180.0 <= chi <= 180.0
constraint: -180.0 <= phi <= 180.0
constraint: -180.0 <= gamma <= 180.0
constraint: -180.0 <= delta <= 180.0
h=2.0 k=1.0 l=

In [22]:
run = cat[uid]
run.metadata["start"]["diffractometers"]["e6c_hkl"]
run

BlueskyRun
  uid='634c4056-3c84-45f0-9ea4-ff08a7fed38c'
  exit_status='success'
  2025-02-10 17:24:40.784 -- 2025-02-10 17:24:40.960
  Streams:
    * primary


-----

## TODO:

Add more content:

- Use `databroker.temp()` to enable plotting.
- plotting from `dev_demo_psi_scan_hklpy2.ipynb`
- `zz_hklpy-demo_psi_scan.ipynb`